# Deep and Reinforcement Learning 2024/2025 (M.IA003), FEUP/FCUP

## Deep Learning Project 
## **Develop deep learning discriminative and generative models, applied to the context of “deep fakes”**

work done by:
- Michal Kowalski up
- Pedro Pereira up
- Pedro Azevedo up201905966

## 1.1) Import Necessary

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import glob
from _model import build_gan, train_gan, train_step
from tensorflow.keras.models import load_model
import json


print(tf.config.list_physical_devices('GPU'))
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
# Define dataset directories
data_dir = "data"
real_dir = os.path.join(data_dir, "real")
fake_dir = os.path.join(data_dir, "fake")

## 1.2) Basic EDA

In [ ]:
from _eda import base_stats

base_stats(real_dir, fake_dir)

## 1.3) File Standardization 

In [ ]:
from _preprocess import standardize_files

standardize_files('data/real', (64, 64), '64')
standardize_files('data/fake', (64, 64), '64')

In [ ]:
real_dir = os.path.join(data_dir, "real_normalized_64")
fake_dir = os.path.join(data_dir, "fake_normalized_64")
base_stats(real_dir, fake_dir)

## 1.4) Baseline

In [ ]:
d, g, do, go = build_gan()

In [ ]:

batch_size = 256
# Get all image file paths

def save_paths(path_list, fname):
    with open(fname, "w") as final:
	    json.dump(path_list, final)

# Define the preprocessing function
def load_and_preprocess(image_path):
    # Read the image file
    image = tf.io.read_file(image_path)

    # Decode the JPEG image
    image = tf.image.decode_jpeg(image, channels=3)

    # Normalize the image to the range [-1, 1]
    image = (tf.cast(image, tf.float32) / 127.5) - 1.0

    return image

def load_paths(train_file, test_file):
    with open(train_file, "r") as train:
	    train_paths = json.load(train)
    with open(test_file, "r") as test:
	    test_paths = json.load(test)
    return train_paths, test_paths

# image_paths = glob.glob("./data/real_normalized_64/*.jpg")
# train_paths = image_paths[:25000]
# test_paths = image_paths[25000:]
# save_paths(train_paths, 'train.json')
# save_paths(test_paths, 'test.json')

train_paths, test_paths = load_paths('./train.json', './test.json')

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices(train_paths)
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size//2, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
d.summary()

In [ ]:
g.summary()

In [ ]:
d_losses, e_losses = train_gan(dataset, d, g, do, go, epochs=1, batch_size=batch_size)

In [ ]:
g.save('my_generator.keras')
d.save('my_discriminator.keras')

In [ ]:
final_d_losses = [float(x) for x in d_losses]
final_g_losses = [float(x) for x in e_losses]
with open('d_losses.json', "w") as f:
    json.dump(final_d_losses, f)

with open('g_losses.json', "w") as f:
    json.dump(final_g_losses, f)

In [ ]:
d = load_model('./my_discriminator200.keras')
g = load_model('./my_generator200.keras')

go = tf.keras.optimizers.Adam(1e-4)
do = tf.keras.optimizers.Adam(1e-4)

In [ ]:
g = saved_model.layers[1]
g.summary()

In [ ]:
d = saved_model.layers[2]
gan = saved_model

In [ ]:
d_losses, e_losses = train_gan(dataset, d, g, do, go, epochs=200, batch_size=batch_size)